In [49]:
import geopandas as gpd
import partridge as ptg
# import modin.pandas as pd
import pandas as pd
idx = pd.IndexSlice
import numpy as np

import requests
from arcgis2geojson import arcgis2geojson

from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
#from folium.element import IFrame
import shapely
#from shapely.geometry import Point
import pysal as ps
from pysal.viz import mapclassify

import os
import datetime as dt

from tqdm.auto import tqdm
tqdm.pandas(desc="interpolating stops...")

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [24]:
def diag_path(path):
    print(ptg.read_busiest_date(path))
    return feed_from_path(path)

In [25]:
def showall(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)

In [26]:
#read feed at a different date if busiest date not in analysis range
#(e.g. post-COVID feed still includes pre-COVID info)
serv_id_exceptions = {'pasadena20200806_gtfs.zip': [3]}

def feed_from_path(path):
    '''Using Partridge, read a (GeoPandas enabled) GTFS feed given a filepath'''
    _date, service_ids = ptg.read_busiest_date(path)
    
    #check if feed has exception defined above, if so read that service id
    feed_id = path.split('/')[-2:][0] + path.split('/')[-2:][1]
    try:
        service_ids = serv_id_exceptions[feed_id]
    except:
        pass
    
    view = {
        'trips.txt': {'service_id': service_ids},
    }

    return ptg.load_geo_feed(path, view)

In [27]:
def feeds_from_files(folder_path):
    '''
    Given path to a folder structured: folder_path/agency/yyyymmdd_gtfs.zip,
    read all feeds into a nested dict with keys being agency name, then datetime object'''
    feeds = {}
    subdirs = [x[0] for x in os.walk(folder_path)]
    for subdir in subdirs[1:]:
        agency = subdir.split('/')[-1]
        feeds[agency] = {}
        for feed in os.listdir(subdir):
            if feed[0] == '.':
                continue
            datestr = feed.split('_')[0]
            date = dt.datetime.strptime(datestr,'%Y%m%d')
            feeds[agency][date] = feed_from_path(subdir+'/'+feed)
    return feeds

In [28]:
feeds_dict = feeds_from_files('./gtfs_feeds/la/')

In [29]:
feeds_dict

{'torrance': {datetime.datetime(2020, 7, 29, 0, 0): <partridge.gtfs.Feed at 0x147a2a130>,
  datetime.datetime(2020, 1, 6, 0, 0): <partridge.gtfs.Feed at 0x147a2ca60>},
 'lacmta_rail': {datetime.datetime(2020, 2, 29, 0, 0): <partridge.gtfs.Feed at 0x147a1c190>,
  datetime.datetime(2020, 8, 6, 0, 0): <partridge.gtfs.Feed at 0x147a1cf10>},
 'bbb': {datetime.datetime(2020, 6, 5, 0, 0): <partridge.gtfs.Feed at 0x147a1f760>,
  datetime.datetime(2020, 2, 17, 0, 0): <partridge.gtfs.Feed at 0x14007c730>},
 'pvpta': {datetime.datetime(2020, 7, 30, 0, 0): <partridge.gtfs.Feed at 0x1479c75e0>,
  datetime.datetime(2019, 9, 21, 0, 0): <partridge.gtfs.Feed at 0x147921400>},
 'ccb': {datetime.datetime(2020, 1, 14, 0, 0): <partridge.gtfs.Feed at 0x147a22d90>,
  datetime.datetime(2020, 4, 11, 0, 0): <partridge.gtfs.Feed at 0x147a1cb80>},
 'lacmta_bus': {datetime.datetime(2020, 1, 16, 0, 0): <partridge.gtfs.Feed at 0x148594e50>,
  datetime.datetime(2020, 8, 30, 0, 0): <partridge.gtfs.Feed at 0x1479ca070>

In [30]:
def get_county_geog(county):
    '''Using TIGERweb API, get geographies for all tracts within a county'''
    api_url = f'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2019/MapServer/8/query?f=json&outsr=4326&where=STATE={county[:2]}%20and%20county={county[2:]}'
    data = requests.get(api_url).json()
    data = arcgis2geojson(data)
    #print(data)
    gdf = gpd.GeoDataFrame.from_features(data['features'])
    gdf['county'] = county
    gdf.crs = 'EPSG:4326'
    gdf = gdf.rename(columns={'BASENAME':'tract'})
    return gdf

def get_region_geog(counties):
    region = get_county_geog(counties[0])
    for county in counties[1:]:
        region = region.append(get_county_geog(county))
    return region

In [222]:
# sac_counties = ['06113', '06101', '06115', '06061', '06057', '06005', '06017', '06095']
# sac_tahoe = get_region_geog(sac_counties)
# sac_tahoe.to_file("./geographies/sac_tahoe.geojson", driver='GeoJSON')

In [9]:
la_county = gpd.read_file('./geographies/la_county.geojson')

In [11]:
example_feed = feeds_dict['torrance'][dt.datetime(2020, 7, 29, 0, 0)]

In [32]:
## https://gist.github.com/csb19815/476335cb299ddb3d5a1a4b898424bb35

def service_hours(stop_times, time_range=None):
    '''return sum of duration of all trips in stop_times (filtered elsewhere)'''
    
    #support arbitrary time selections...
    if time_range:
        stop_times = (
            stop_times[(stop_times['arrival_time'] >= time_range[0])
            & (stop_times['arrival_time'] <= time_range[1])])
    try:
        trip_lengths = stop_times.groupby('trip_id').arrival_time.agg(['min', 'max'])
        service_hours = (trip_lengths['max'] - trip_lengths['min']) / 60 / 60
        return service_hours.sum()
    except:
        print('serv_hr_error, returning 0')
        return 0


In [15]:
example_feed.stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,374020,18900.0,18900.0,881,1,NaN,0,1,0.00,1
1,374020,19080.0,19080.0,366,2,NaN,0,0,1044.98,1
2,374020,19202.0,19202.0,367,3,NaN,0,0,2292.11,0
3,374020,19307.0,19307.0,368,4,NaN,0,0,3345.81,0
4,374020,19380.0,19380.0,354,5,NaN,0,0,4004.75,1
...,...,...,...,...,...,...,...,...,...,...
17469,123020,81013.0,81013.0,94,47,NaN,0,0,19701.56,0
17470,123020,81106.0,81106.0,95,48,NaN,0,0,20072.45,0
17471,123020,81205.0,81205.0,96,49,NaN,0,0,20506.47,0
17472,123020,81301.0,81301.0,97,50,NaN,0,0,20896.43,0


In [33]:
def interpolate_stops(gdf, geo):
    '''
    When the next stop is in a different tract, estimate when it crosses tracts and add
    that arrival time to both tracts. Important because the next step is to group by tract
    and estimate service hours, service crossing tracts would be dropped without this interpolation.
    '''
    
    #can't interpolate on single-stop trip... (should fix LADOT bug...)
    if gdf.shape[0] < 2:
        return gdf
#     if gdf['geometry'].is_unique == False:
#         print('non-unique stops!')
#         return gdf
    
    gdf.loc[:, 'geometry'] = gdf.geometry.centroid
    gdf.loc[:, 'lasttract'] = gdf['tract'].shift()
    gdf.loc[:, 'lastgeo'] = gpd.GeoSeries(gdf['geometry'].shift())
    gdf.loc[:, 'lastarr'] = gdf['arrival_time'].shift()
    gdf.loc[:, 'sametract'] = gdf['tract'].eq(gdf['lasttract'])
    #debug
#     return gdf
    #reset in case stop_id not a unique index
    gdf = gdf.reset_index()
#     return gdf
    #create a line between every pair of stops
#     print('lining...', end='')
    gdf['line_last_stop'] = gdf.iloc[1:,:].apply(
    lambda x: shapely.geometry.LineString(
        [(x.lastgeo.x, x.lastgeo.y),
         (x.geometry.x, x.geometry.y)
        ]), axis=1)
#     print('lined!...', end='')

    #add column with arrival time at tract boundary
    
#     global _debug
#     _debug = gdf
#     display(gdf)

    gdf['split_arr'] = gdf.apply(line_in_tracts, args=(geo,), axis = 1)
    #generate df with tract+arrival time rows to be appended
    records = gdf.apply(add_interpolated_rows, axis=1).dropna().values
    to_append = pd.DataFrame()
    for record in records:
        to_append = to_append.append(pd.DataFrame(record))
    #append interpolated rows to origional stop times gdf
    merged_df = (gdf.append(to_append).
                 reset_index()[['tract', 'arrival_time', 'stop_sequence', 'trip_id']])
    merged_df.loc[:, 'trip_id'] = merged_df['trip_id'][0]
    return merged_df

In [34]:
def line_in_tracts(df, tract_geos):
    '''Measure how much of the distance between 2 stops is in each tract'''
    if df['sametract'] or  np.all(np.isnan(df['line_last_stop'])):
        return
    tract1 = df['lasttract']
    tract2 = df['tract']
    line = df['line_last_stop']
    tract1_polygon = tract_geos[tract_geos['tract'] == tract1]['geometry'].iloc[0]
    tract2_polygon = tract_geos[tract_geos['tract'] == tract2]['geometry'].iloc[0]
    line_tract1 = line.length - line.difference(tract1_polygon).length
    line_tract2 = line.length - line.difference(tract2_polygon).length
    line_total = line_tract1 + line_tract2
    #display(df)
#     print(line_tract1, line_tract2, line_total)
    #list with first value being % of line in tract1, second being % of line in tract 2
    ##note this is only out of the 2-tract total, ignores potential intermediate tracts
    arrival_splits = (np.array([line_tract1/line_total, line_tract2/line_total]) * 
                      (df['arrival_time'] - df['lastarr']))
    return df['lastarr'] + arrival_splits[0]

In [35]:
def add_interpolated_rows(df):
    '''Generate dict with a single set of interpolated rows'''
    if df['split_arr'] and not np.all(np.isnan(df['split_arr'])):
        two_rows = {'tract': [df['tract'], df['lasttract']], 'arrival_time': df['split_arr']}
        return two_rows

In [16]:
feeds_dict

{'torrance': {datetime.datetime(2020, 7, 29, 0, 0): <partridge.gtfs.Feed at 0x146b2da00>,
  datetime.datetime(2020, 1, 6, 0, 0): <partridge.gtfs.Feed at 0x1476a1cd0>},
 'lacmta_rail': {datetime.datetime(2020, 2, 29, 0, 0): <partridge.gtfs.Feed at 0x1475d1640>,
  datetime.datetime(2020, 8, 6, 0, 0): <partridge.gtfs.Feed at 0x147480460>},
 'bbb': {datetime.datetime(2020, 6, 5, 0, 0): <partridge.gtfs.Feed at 0x108cc7fd0>,
  datetime.datetime(2020, 2, 17, 0, 0): <partridge.gtfs.Feed at 0x146bafc10>},
 'pvpta': {datetime.datetime(2020, 7, 30, 0, 0): <partridge.gtfs.Feed at 0x108cc7ee0>,
  datetime.datetime(2019, 9, 21, 0, 0): <partridge.gtfs.Feed at 0x1475d16a0>},
 'ccb': {datetime.datetime(2020, 1, 14, 0, 0): <partridge.gtfs.Feed at 0x146b2d910>,
  datetime.datetime(2020, 4, 11, 0, 0): <partridge.gtfs.Feed at 0x146b7f430>},
 'lacmta_bus': {datetime.datetime(2020, 1, 16, 0, 0): <partridge.gtfs.Feed at 0x146b07dc0>,
  datetime.datetime(2020, 8, 30, 0, 0): <partridge.gtfs.Feed at 0x1478a59a0>

In [17]:
# test_agencies = ['torrance', 'lacmta_rail', 'bbb', 'ccb']
test_agencies = ['torrance']
test_dict = {agency: feeds_dict[agency] for agency in test_agencies}

In [18]:
test_dict

{'torrance': {datetime.datetime(2020, 7, 29, 0, 0): <partridge.gtfs.Feed at 0x146b2da00>,
  datetime.datetime(2020, 1, 6, 0, 0): <partridge.gtfs.Feed at 0x1476a1cd0>}}

In [36]:
def service_hours_by_geo(times_in_geo, serv_type):
    '''Calculate service hours by tract at a particular time of day'''
    #GTFS times are measured in seconds since midnight, so 6*60**2 corresponds to 6:00AM
    serv_times = {'am_peak': (6*60**2, 9*60**2),
                  'midday': (9*60**2, 15*60**2),
                 'pm_peak': (15*60**2, 19*60**2),
                 'evening': (19*60**2, 28*60**2),
                 'early_am': (0*60**2, 6*60**2)}
    
    grouped = pd.DataFrame(times_in_geo.groupby("tract").
            apply(service_hours, time_range = (serv_times[serv_type])))
    grouped = grouped.rename(columns = {0:f'{serv_type}_vrh'})
    
    return grouped

In [45]:
def single_agency_df(feed, geo):
    '''Calculate service hours by tract by service type for a single feed (agency+date)'''
    serv_types = ['am_peak', 'midday', 'pm_peak', 'evening', 'early_am']
    
    #interpolate between stop times if needed
    if feed.stop_times['arrival_time'].isnull().any():
        nulls = feed.stop_times['arrival_time'].isnull().value_counts()[True]
        print(f'Feed contains {nulls} null stop_times entries, interpolating...')
        feed.stop_times['arrival_time'] = feed.stop_times['arrival_time'].interpolate()
        feed.stop_times['departure_time'] = feed.stop_times['departure_time'].interpolate()
        
    def stops_in_geo(feed, geo):

        stops_in_geo = gpd.sjoin(geo, feed.stops,
                                 how='inner', op='intersects')
        stops_in_geo = stops_in_geo.drop_duplicates(subset=['stop_id'])
        stops_in_geo = stops_in_geo.set_index('stop_id')
        return stops_in_geo

    times_in_geo = stops_in_geo(feed, geo).join(
        feed.stop_times.set_index('stop_id'), how='inner')
    ##BBB debug return
#     return times_in_geo

    #interpolate_stops is applied here once per feed (this is the slowest step)
#     times_in_geo = (times_in_geo.sort_values(by=['trip_id', 'stop_sequence'])
#                .groupby('trip_id').apply(interpolate_stops, geo=geo)
#                .rename(columns={'trip_id':'trip_id2'}))

    times_in_geo = (times_in_geo.sort_values(by=['trip_id', 'stop_sequence'])
               .groupby('trip_id').progress_apply(interpolate_stops, geo=geo)
               .rename(columns={'trip_id':'trip_id2'}))

    vrh_by_serv_type = pd.DataFrame()
    for serv_type in serv_types:
        print(serv_type)
        vrh_by_serv_type = vrh_by_serv_type.append(
            service_hours_by_geo(times_in_geo, serv_type))
    return vrh_by_serv_type

In [38]:
def service_hour_df(feeds_dict, geo):
    '''
    Given dictionary of feeds generated in previous step,
    generate final dataframe with VRH by tract, COVID status, and agency.
    Currently slow to run if feeds are large (expect 20-40 minutes for LA area).
    '''
    mdf = pd.DataFrame()
    
    for agency in list(feeds_dict.keys()):
        print(agency)
        agency_dict = feeds_dict[agency]
        pre_covid = agency_dict[min(agency_dict.keys())]
        pre_covid.stops.crs = 'EPSG:4326'
        current = agency_dict[max(agency_dict.keys())]
        current.stops.crs = 'EPSG:4326'
        try:
            print('pre-covid')
            ##TODO add some sort of progress print in interpolate_stops?
            pre_covid_hrs = single_agency_df(pre_covid, geo)
            pre_covid_hrs['covid'] = 0
            pre_covid_hrs.set_index('covid', append=True, inplace=True)
    #         return pre_covid_hrs
            print('covid')
            current_hrs = single_agency_df(current, geo) 
            current_hrs['covid'] = 1
            current_hrs.set_index('covid', append=True, inplace=True)
        #         return current_hrs
        #skip to next agency if either of these fail (continuing to debug interpolation)...
        except:
            continue
        
        agency_hrs = pre_covid_hrs.append(current_hrs)
        agency_hrs['agency'] = agency
        agency_hrs.set_index('agency', append=True, inplace=True)
        mdf = mdf.append(agency_hrs)
        
    mdf = mdf.groupby(level=['tract', 'covid', 'agency']).sum()
    mdf['total_vrh'] = mdf.sum(axis=1)
    return mdf

In [52]:
svc_torrance = service_hour_df(test_dict, la_county)

torrance
pre-covid



am_peak
midday
pm_peak
evening
early_am
covid



am_peak
midday
pm_peak
evening
early_am


In [53]:
svc_torrance

am_peak_vrh  midday_vrh  pm_peak_vrh  evening_vrh  \
tract   covid agency                                                        
2060.20 0     torrance     0.523667    0.089840     0.458807     0.000000   
        1     torrance     0.278967    0.081886     0.332347     0.000000   
2071.02 0     torrance     0.603654    0.166207     1.159037     0.000000   
        1     torrance     0.446120    0.146900     0.633898     0.139722   
2074    0     torrance     0.747769    0.170736     1.156497     0.000000   
...                             ...         ...          ...          ...   
9800.13 1     torrance     0.894872    1.659976     0.969670     0.901833   
9800.14 0     torrance     0.370122    0.847625     0.334478     0.147550   
        1     torrance     0.276063    0.483111     0.310094     0.118040   
9800.28 0     torrance     1.605516    3.165642     2.008639     1.301553   
        1     torrance     0.748047    1.319018     0.768831     0.891725   

                        early_am_vrh  total_vrh  
tract   covid agency                             
2060.20 0     torrance      0.000000   1.072315  
        1     torrance      0.000000   0.693200  
2071.02 0     torrance      0.000000   1.928898  
        1     torrance      0.000000   1.366640  
2074    0     torrance      0.000000   2.075003  
...                              ...        ...  
9800.13 1     torrance      0.186314   4.612665  
9800.14 0     torrance      0.103524   1.803299  
        1     torrance      0.034508   1.221816  
9800.28 0     torrance      0.240810   8.322160  
        1     torrance      0.073725   3.801346  

[259 rows x 6 columns]

In [284]:
svc_sac_tahoe.loc[idx[:,:,'Sacramento RT'],:]

am_peak_vrh  midday_vrh  pm_peak_vrh  evening_vrh  \
tract  covid agency                                                             
105.01 1     Sacramento RT     0.687778        1.05     0.983951         0.35   
105.05 1     Sacramento RT     0.178888        0.00     0.099382         0.00   
207.14 0     Sacramento RT     0.000000        0.00     0.000000         0.00   
       1     Sacramento RT     0.000000        0.00     0.000000         0.00   
208.05 0     Sacramento RT     0.000000        0.00     0.000000         0.00   
       1     Sacramento RT     0.000000        0.00     0.000000         0.00   

                            early_am_vrh  total_vrh  
tract  covid agency                                  
105.01 1     Sacramento RT      0.083333   3.155063  
105.05 1     Sacramento RT      0.000000   0.278271  
207.14 0     Sacramento RT      0.000000   0.000000  
       1     Sacramento RT      0.000000   0.000000  
208.05 0     Sacramento RT      0.000000   0.000000  
       1     Sacramento RT      0.000000   0.000000

In [262]:
_debug

,stop_id,geometry,tract,county,index_right,stop_code,platform_code,stop_name,stop_desc,zone_id,...,continuous_drop_off,pickup_area_id,drop_off_area_id,pickup_service_area_radius,drop_off_service_area_radius,lasttract,lastgeo,lastarr,sametract,line_last_stop
0,19037,POINT (-120.57074 38.46494),1.01,06005,41,NaN,NaN,Amador Station,NaN,770,...,1,NaN,NaN,NaN,NaN,NaN,None,NaN,False,NaN
1,19033,POINT (-120.57074 38.46494),1.01,06005,38,NaN,NaN,Mace Meadows / Sugar Pines,NaN,770,...,1,NaN,NaN,NaN,NaN,1.01,POINT (-120.57074 38.46494),23520.0,True,LINESTRING (-120.5707395972678 38.464944556116...
2,19030,POINT (-120.25532 38.54226),1.02,06005,36,NaN,NaN,Silver Drive,NaN,770,...,1,NaN,NaN,NaN,NaN,1.01,POINT (-120.57074 38.46494),24060.0,False,LINESTRING (-120.5707395972678 38.464944556116...
3,19029,POINT (-120.25532 38.54226),1.02,06005,35,NaN,NaN,IGA Village Market,NaN,770,...,1,NaN,NaN,NaN,NaN,1.02,POINT (-120.25532 38.54226),24180.0,True,LINESTRING (-120.2553223367905 38.542260582521...
4,19027,POINT (-120.25532 38.54226),1.02,06005,34,NaN,NaN,Buckhorn Ridge Loop (Pioneer Park),NaN,770,...,1,NaN,NaN,NaN,NaN,1.02,POINT (-120.25532 38.54226),24360.0,True,LINESTRING (-120.2553223367905 38.542260582521...
5,19026,POINT (-120.25532 38.54226),1.02,06005,33,NaN,NaN,Pioneer Post Office,NaN,770,...,1,NaN,NaN,NaN,NaN,1.02,POINT (-120.25532 38.54226),24720.0,True,LINESTRING (-120.2553223367905 38.542260582521...
6,19022,POINT (-120.66037 38.39478),4.02,06005,29,NaN,NaN,Kamps Propane,NaN,770,...,1,NaN,NaN,NaN,NaN,1.02,POINT (-120.25532 38.54226),24840.0,False,LINESTRING (-120.2553223367905 38.542260582521...
7,27802,POINT (-120.66037 38.39478),4.02,06005,67,NaN,NaN,Homestead Rd.,NaN,770,...,1,NaN,NaN,NaN,NaN,4.02,POINT (-120.66037 38.39478),25080.0,True,LINESTRING (-120.6603692946687 38.394781260730...
8,27803,POINT (-120.66037 38.39478),4.02,06005,68,NaN,NaN,Gayla Dr.,NaN,770,...,1,NaN,NaN,NaN,NaN,4.02,POINT (-120.66037 38.39478),25140.0,True,LINESTRING (-120.6603692946687 38.394781260730...
9,19020,POINT (-120.66037 38.39478),4.02,06005,27,NaN,NaN,Pine Acres,NaN,769,...,1,NaN,NaN,NaN,NaN,4.02,POINT (-120.66037 38.39478),25200.0,True,LINESTRING (-120.6603692946687 38.394781260730...


In [281]:
svc_sac_tahoe.to_json('./processed_data/sac_tahoe/sac_tahoe_service.json', orient='table')

## To-do:

### Feature Adds:
* _visualization along routes_
* geoid, not tracts?
* "megaregion" view?? --> likely requires automating feed collection...
    * also some sort of tiling, aggregation by city, etc...
    
### Architecture
* make compatible with Modin?
